In [1]:
import numpy as np
import matplotlib.pyplot as plt
from model import run_model
from params import par_class

In [2]:
# Data
THs = [27-25, 42-25, 57-25]
zs  = [0.937, 0.802, 0.751]

In [3]:
# Function for computing simulated moments
def get_zs_sim(gamma):
    beta = gamma[0]
    rho = gamma[1]
    par = par_class()
    par.Nm = 80
    par.Nh = 12
    par.simN = 250_000
    par.beta = beta
    par.rho = rho
    zs_sim = []
    for TH in THs:
        par.TH = TH
        _, sim = run_model(par)
        z_mean = sim.z.mean()
        zs_sim.append(z_mean)
    mean_squared_dist = np.mean((np.array(zs) - np.array(zs_sim))**2)
    print(f'beta = {beta:.4f}, rho={rho:.4f}, and objective = {mean_squared_dist:.10f}')
    return np.array(zs_sim)

# Compute simulated moments around estimator (to compute numerical gradient later)
h1 = 0.0005 # Step size for beta
h2 = 0.005  # Step size for rho
zs_sim = np.nan + np.zeros((3,3,3))
for i, beta in enumerate([0.99-h1, 0.99, 0.99+h1]):
    for j, rho in enumerate([4.8-h2, 4.8, 4.8+h2]):
        zs_sim[i,j,:] = get_zs_sim([beta,rho])

beta = 0.9895, rho=4.7950, and objective = 0.0143601640
beta = 0.9895, rho=4.8000, and objective = 0.0143555478
beta = 0.9895, rho=4.8050, and objective = 0.0143568311
beta = 0.9900, rho=4.7950, and objective = 0.0143850978
beta = 0.9900, rho=4.8000, and objective = 0.0143700730
beta = 0.9900, rho=4.8050, and objective = 0.0143595088
beta = 0.9905, rho=4.7950, and objective = 0.0144365393
beta = 0.9905, rho=4.8000, and objective = 0.0144088054
beta = 0.9905, rho=4.8050, and objective = 0.0143845642


In [4]:
# Sample size
N1 = 269_424/0.937
N2 = 244_893/0.802
N3 = 228_937/0.751
S = 250_000
N = np.min([N1,N2,N3])

# G matrix
grad_0 = np.array(np.gradient(zs_sim[:,:,0], h1, h2))[:,1,1]
grad_1 = np.array(np.gradient(zs_sim[:,:,1], h1, h2))[:,1,1]
grad_2 = np.array(np.gradient(zs_sim[:,:,2], h1, h2))[:,1,1]
G = np.nan + np.zeros((3,2))
G[0,:] = grad_0
G[1,:] = grad_1
G[2,:] = grad_2

# W matrix
W = np.eye(3)

# Omega matrix
Omega = np.zeros((3,3))
Omega[0,0] = 0.937*(1-0.937)
Omega[1,1] = 0.802*(1-0.802)
Omega[2,2] = 0.751*(1-0.751)

# V matrix
A_inv = np.linalg.inv(G.T @ W @ G)
B = G.T @ W.T @ Omega @ W @ G
V = A_inv @ B @ A_inv

# Standard errors
se = np.sqrt(np.diag((1/N)*(1+1/S)*V))
print(f'se(beta): {se[0]:.5f}')
print(f'se(rho):  {se[1]:.5f}')

se(beta): 0.07643
se(rho):  1.58300
